##### Parsing Biogrid data

In [1]:
import os
import pandas as pd

directory_path = 'files/construction_data'

file_extension = '.txt'

dataframes = []
enterez_id_pairs = set()

for file in os.listdir(directory_path):
    if file.endswith(file_extension):
        file_path = os.path.join(directory_path, file)
        df = pd.read_csv(file_path, delimiter='\t')
        df = df[~df['Experimental System'].isin(['Co-localization', 'Genetic interference'
                                      'Synthetic Rescue', 'Synthetic Growth Defect',
                                      'Synthetic Lethality'])]
        df = df[(df['Entrez Gene Interactor A'] != '-') & (df['Entrez Gene Interactor B'] != '-')]
        
        dataframes.append(df)
        
#combined_data = pd.concat(dataframes, ignore_index=True)

In [2]:
for df in dataframes:
    for index, row in df.iterrows():
        id_a = row['Entrez Gene Interactor A']
        id_b = row['Entrez Gene Interactor B']

        if int(id_a) != int(id_b):
            pair = tuple(sorted([id_a, id_b]))
            enterez_id_pairs.add(pair)

NameError: name 'unique_pairs' is not defined

In [3]:
print(enterez_id_pairs)

{(1025, 9049), (3362, 9049), (9049, 440730), (9049, 55172), (9049, 54461), (9049, 391004), (604, 9049), (780, 9049), (7957, 9049), (3320, 9049), (2261, 9049), (355, 9049), (9049, 284086), (2023, 9049), (5613, 9049), (9049, 83983), (5166, 9049), (9049, 57407), (3654, 9049), (9049, 10869), (5150, 9049), (5071, 9049), (5894, 9049), (2870, 9049), (7415, 9049), (1844, 9049), (332, 9049), (9049, 9212), (5139, 9049), (9049, 10963), (8517, 9049), (8189, 9049), (9049, 27068), (9049, 80824), (4914, 9049), (9049, 83942), (9049, 348793), (9049, 25939), (5979, 9049), (9049, 43740578), (8915, 9049), (9049, 23678), (2046, 9049), (9049, 10910), (7328, 9049), (9049, 26094), (1161, 9049), (5609, 9049), (9049, 342898), (9049, 29941), (9049, 26469), (9049, 55626), (9049, 80344), (8798, 9049), (8481, 9049), (9049, 25822), (9049, 9927), (3916, 9049), (2776, 9049), (1198, 9049), (9049, 10110), (3665, 9049), (1781, 9049), (9049, 55827), (9049, 112399), (7345, 9049), (9049, 9344), (1968, 9049), (9049, 78995), 

In [44]:
len(dataframes[0])

235

In [17]:
data.columns

Index(['#BioGRID Interaction ID', 'Entrez Gene Interactor A',
       'Entrez Gene Interactor B', 'BioGRID ID Interactor A',
       'BioGRID ID Interactor B', 'Systematic Name Interactor A',
       'Systematic Name Interactor B', 'Official Symbol Interactor A',
       'Official Symbol Interactor B', 'Synonyms Interactor A',
       'Synonyms Interactor B', 'Experimental System',
       'Experimental System Type', 'Author', 'Publication Source',
       'Organism ID Interactor A', 'Organism ID Interactor B', 'Throughput',
       'Score', 'Modification', 'Qualifications', 'Tags', 'Source Database',
       'SWISS-PROT Accessions Interactor A', 'TREMBL Accessions Interactor A',
       'REFSEQ Accessions Interactor A', 'SWISS-PROT Accessions Interactor B',
       'TREMBL Accessions Interactor B', 'REFSEQ Accessions Interactor B',
       'Ontology Term IDs', 'Ontology Term Names', 'Ontology Term Categories',
       'Ontology Term Qualifier IDs', 'Ontology Term Qualifier Names',
       'Ontology

In [4]:
import mygene

mg = mygene.MyGeneInfo()

ids = ['140876']  # Ovo je Entrez ID za gene POTEF

result = mg.querymany(ids, scopes='entrezgene', fields='symbol', species='human')

for gene_info in result:
    symbol = gene_info.get('symbol', 'No symbol found')
    print(f"ID: {gene_info['query']} maps to gene symbol: {symbol}")

ID: 140876 maps to gene symbol: RIPOR3
